# Manual 03: Optimización Max-Min
## Universidad del Norte
### Economía Matemática
Llegó la parte de **optimización** a la clase. Vamos hacer algunos ejercicios a mano primero con las funciones propias y luego si nos vamos a paquetes que permitan obtener resultados mas concretos.

Empecemos primero con el analisis de la tradicional función de Cobb-Douglas:
$$ \max_{x_1, x_2} \quad x_1^{\alpha} x_2^{\beta} \quad \text{s.a.} \quad p_1 x_1 + p_2 x_2 = m $$

En repetidas ocasiones -*sobre todo en Microeconomía*-, se solicitaba mediante el proceso de Lagrange, hallar cada uno de los valores óptimos que máximizaban una función de utilidad y que estaba bajo una restricción especifica.

Ademas que despues de aplicar la intercepción de nuestros precios relativos con las Utilidades Marginales se lograba una ecuación que permitía incorporarse en la restricción y con ello encontrar la solución de:

$$ x_1^* = \frac{\alpha}{\alpha + \beta} \frac{m}{p_1}, \quad x_2^* = \frac{\beta}{\alpha + \beta} \frac{m}{p_2} $$

Lo anterior ampliamente conocido como <span class="rojo">Demandas Marshallianas</span>. Ya que estas eran *observables* y dependian directamente del ingreso pero inversamente de los precios de los bienes.

$$ x_1^* = \frac{\alpha m}{p_1}, \quad x_2^* = \frac{(1-\alpha) m}{p_2} $$

## Trabajo en python

La idea es entonces hacerlo en python. Para ello creemos una función

In [ ]:
def Marshalliana(alpha, m, p1, p2):
    x1 = alpha * m / p1
    x2 = (1 - alpha) * m / p2
    return x1, x2

Digamos por un momento que la función tendrá los siguientes valores 
$$\alpha = 0.5, \ m = 200, \ p_1 = 7, \ p_2 = 15$$
Que es lo mismo que $x_1^* = 0.5 \times \frac{200}{7} = 14,28$ y $x_2^* = 0.5 \times \frac{200}{15} = 6,6$.

In [ ]:
# Parámetros
alpha = 0.5
m = 200
p1 = 7
p2 = 15

# Haga uso de la funcióm
x1, x2 = Marshalliana(alpha, m, p1, p2)

# Dandole confort al resultado y redondeando decimales
print(f"Demanda del bien 1: {round(x1, 2)}")
print(f"Demanda del bien 2: {round(x2, 2)}")

### Estatica comparativa
Vamos a realizar un gráfico del bien $x_1$ y luego vamos a mirar que ocurre si aumenta un 15% el nivel de ingreso de la persona

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Vector donde va los precios
p1_vals = np.linspace(1, 10, 25)

# Se resuelve por cada punto
x1_vals, x2_vals = Marshalliana(alpha, m, p1_vals, p2)

# Grafica
plt.plot(p1_vals, x1_vals)
plt.xlabel('Precio del Bien 1')
plt.ylabel('Demanda del Bien 1')
plt.show()

### Pensemos
Que ocurre si el presupuesto aumenta en un 15% $I\rightarrow 15\%$ entonces es $200 \cdot (1+0.15)= 230$. Que va ocurrir?.

In [ ]:
# Calcular la demanda cuando I = 230
x1_vals2, x2_vals2 = Marshalliana(alpha, 230, p1_vals, p2)

# Create a new plot with the second curve
plt.plot(p1_vals, x1_vals, label='m = 200')
plt.plot(p1_vals, x1_vals2, label='m = 230')
plt.xlabel('Precio del Bien 1')
plt.ylabel('Demanda del Bien 1')
plt.legend()
plt.show()

Note que la función optimiza en varios puntos al cambiar los parámetros. Intente usted cambiar al ingreso a $500 y vea como aumenta aun mas la expresión.

## Optimización Automática
llegó la hora de optimizar con una función del paquete `scipy`. Aunque aparezca `minimize`, debemos setear de tal manera que sea plausible la respuesta de nuestra función objetivo.

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Parameters
alpha = 0.5
m = 200
p1 = 7
p2 = 15

# Funciones de Utilidad
def utilidad(x):
    x1, x2 = x
    if x1 < 0 or x2 < 0:
        return np.inf
    else:
        return -x1 ** alpha * x2 ** (1 - alpha)

# Restricción Presupuesto
def presupuesto_c(x):
    x1, x2 = x
    return m - p1 * x1 - p2 * x2

# Consolidando las restricciones
constraints = ({"type": "eq", "fun": presupuesto_c})
initial_guess = [1, 2] 

# Aplicando la funciín -f(x) para maximizar
resultado = minimize(utilidad, initial_guess, constraints=constraints)
x1, x2 = resultado.x

# Print the results (round to 2 decimal places)
print(f"Demanda del Bien 1: {round(x1, 2)}")
print(f"Demanda del Bien 2: {round(x2, 2)}")

### Función CES

Digamos que queremos ser mas avanzados y usar dos restricciones y optimizar una función de utilidad tipo CES o (Constant Elasticity Sustitution).

$$U(x_1, x_2) = \left( x_1^\rho + x_2^\rho \right)^{\frac{1}{\rho}}$$

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Parámetros
rho = 0.5
m = 300
p1 = 12
p2 = 18

# Función de utilidad CES
def utilidad(x):
    x1, x2 = x
    if x1 < 0 or x2 < 0:
        return np.inf
    else:
        return -((x1 ** rho + x2 ** rho) ** (1 / rho))

# Restricción presupuestaria
def presupuesto_c(x):
    x1, x2 = x
    return m - p1 * x1 - p2 * x2

# Restricción adicional: cantidad mínima de bien 2
def restriccion_minima_bien2(x):
    x1, x2 = x
    return x2 - 3  # x2 debe ser al menos 3

# Especificación de restricciones y suposición inicial
constraints = ({"type": "eq", "fun": presupuesto_c},
               {"type": "ineq", "fun": restriccion_minima_bien2})
initial_guess = [5, 4]

# Minimización de la función de utilidad
resultado = minimize(utilidad, initial_guess, constraints=constraints)
x1, x2 = resultado.x

# Impresión de resultados (redondeados a 2 decimales)
print(f"Demanda del Bien 1: {round(x1, 2)}")
print(f"Demanda del Bien 2: {round(x2, 2)}")

### Ejercicio

1. Replique la maximización pero hagalo cuando es una demanda de bienes sustitutos.
2. ¿Cómo afecta la suposición inicial a la convergencia del algoritmo? Prueba con diferentes suposiciones iniciales como [3,4] y [6,7].